In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.9 MB/s eta 0:00:00


In [186]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from numpy import mean
from numpy import std
import string
import math

In [274]:
casas = pd.read_csv("kc_house_data.csv")

casas.drop(['id'],axis=1,inplace=True)
casas.drop(['date'],axis=1,inplace=True)

In [225]:
# O TRECHO COMENTADO ABAIXO NÃO MELHOROU OS RESULTADOS

# Aplicando o OneHotEncoder de forma mais simples
#casas = pd.get_dummies(casas, columns=['zipcode'], prefix='zipcode')

In [282]:
multiplicador = 0.01

def identificar_outliers(coluna):
    Q1 = coluna.quantile(0.25)
    Q3 = coluna.quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - multiplicador * IQR
    limite_superior = Q3 + multiplicador * IQR
    outliers = coluna[(coluna < limite_inferior) | (coluna > limite_superior)]
    return outliers

for coluna in casas.columns:
    if pd.api.types.is_numeric_dtype(casas[coluna]):
        outliers = identificar_outliers(casas[coluna])

        if not outliers.empty:
            print(f"Outliers na coluna {coluna}:\n{outliers}\n")

Outliers na coluna price:
0         221900.0
2         180000.0
5        1225000.0
6         257500.0
7         291850.0
           ...    
21597    1575000.0
21599     810000.0
21600    1537000.0
21602     224000.0
21606    1007500.0
Name: price, Length: 10583, dtype: float64

Outliers na coluna bedrooms:
1164     5
1315     6
1448     5
3914     5
4411     5
7252     6
8092     5
9254     6
12370    5
Name: bedrooms, dtype: int64

Outliers na coluna bathrooms:
2        1.00
13       1.75
18       1.00
20       1.75
26       1.75
         ... 
21530    3.75
21534    1.75
21572    1.75
21576    4.50
21602    1.75
Name: bathrooms, Length: 3632, dtype: float64

Outliers na coluna sqft_living:
0        1736
2        1688
4        1680
5        2154
6        1715
         ... 
21605    2520
21606    2160
21608    1530
21609    2310
21611    1600
Name: sqft_living, Length: 10641, dtype: int64

Outliers na coluna sqft_lot:
0         5650
1         7242
2        10000
3         5000
4        

In [276]:
# Colunas que irá passar por ajustes de outliers (0 - não, 1 - sim)
colunas_avaliadas = {
    'price': 0,
    'bedrooms': 1,
    'bathrooms': 1,
    'sqft_living': 1,
    'sqft_lot': 1,
    'floors': 1,
    'waterfront': 0,
    'view': 0,
    'condition': 0,
    'grade': 0,
    'sqft_above': 1,
    'sqft_basement': 1,
    'yr_built': 0,
    'yr_renovated': 0,
    'zipcode': 0,
    'lat': 0,
    'long': 0,
    'sqft_living15': 1,
    'sqft_lot15': 1
}

multiplicador = 0.01 # Comecei em 1.5 e fui diminuindo, quanto menor o valor mais lento é para ajustar os outliers, no entanto, melhora o resultado.
qtd = 0

for coluna in casas.columns:
    if pd.api.types.is_numeric_dtype(casas[coluna]) and colunas_avaliadas.get(coluna, 0):
        Q1 = casas[coluna].quantile(0.25)
        Q3 = casas[coluna].quantile(0.75)
        IQR = Q3 - Q1

        # Calcular os limites para identificação de outliers
        limite_inferior = Q1 - multiplicador * IQR
        limite_superior = Q3 + multiplicador * IQR

        outliers = (casas[coluna] < limite_inferior) | (casas[coluna] > limite_superior)

        for outlier in casas[coluna][outliers].index:
            preco_correspondente = casas.loc[outlier, 'price']

            # Considerar uma tolerância de 50 mil para mais ou para menos no preço correspondente
            preco_minimo = preco_correspondente - 50000
            preco_maximo = preco_correspondente + 50000

            valores_nao_outliers = casas[(casas['price'] >= preco_minimo) & (casas['price'] <= preco_maximo) & ~outliers][coluna]

            # Verificar se há valores não outliers para calcular a média
            if not valores_nao_outliers.empty:
                media_valor = valores_nao_outliers.mean()
                casas.loc[outlier, coluna] = int(media_valor)
            else:
                # Se não houver valores não outliers, incrementar a contagem
                #casas = casas.drop(outlier)
                qtd += 1

print(qtd)

518


In [277]:
imputer = KNNImputer(n_neighbors=5)
casas[['sqft_basement', 'yr_renovated']] = imputer.fit_transform(casas[['sqft_basement', 'yr_renovated']])

In [278]:
casas['view'] = casas['view'].apply(lambda x: 1 if x > 0 else 0)
casas['yr_renovated'] = casas['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)
casas['waterfront'] = casas['waterfront'].apply(lambda x: 1 if x > 0 else 0)

# NÃO MELHOROU O RESULTADO, FEZ PIORAR
#casas['sqft_basement'] = casas['sqft_basement'].apply(lambda x: 1 if x > 0 else 0)


In [210]:
#A PARTE COMENTADA ABAIXO AUMENTOU O ERRO

#casas = casas.drop_duplicates()

#condition = (casas['sqft_living'] > casas['sqft_lot']) & (casas['floors'] == 1)
#for index, row in casas[condition].iterrows():
#    casas.at[index, 'sqft_living'] = row['sqft_living15']
#    casas.at[index, 'sqft_lot'] = row['sqft_lot15']

#condition = (casas['sqft_living15'] > casas['sqft_lot15']) & (casas['floors'] == 1)
#for index, row in casas[condition].iterrows():
#    casas.at[index, 'sqft_living15'] = row['sqft_living']
#    casas.at[index, 'sqft_lot15'] = row['sqft_lot']

#def classificar_regiao(lat, long, num_quadrantes):
#    # Define as coordenadas dos limites dos quadrantes
#    lat_limites = [47.1559 + i * ((47.7776 - 47.1559) / num_quadrantes) for i in range(num_quadrantes + 1)]
#    long_limites = [-122.519 + i * ((-121.315 - (-122.519)) / num_quadrantes) for i in range(num_quadrantes + 1)]

#    # Função para obter a letra do quadrante
#    def obter_letra(coordenadas, valor):
#        for i, limite in enumerate(coordenadas):
#            if valor < limite:
#                return string.ascii_uppercase[i]
#        return string.ascii_uppercase[-1]

    # Calcula as letras dos quadrantes para lat e long
#    lat_letra = obter_letra(lat_limites, lat)
#    long_letra = obter_letra(long_limites, long)

#    # Combina as letras para formar a região
#    regiao = f"{lat_letra}{long_letra}"

#    return regiao

# Número de quadrantes desejado
#num_quadrantes = 90
#num_quadrantes = int(math.sqrt(num_quadrantes))

# Aplica a função para criar uma nova coluna 'regiao'
#casas['regiao'] = casas.apply(lambda row: classificar_regiao(row['lat'], row['long'], num_quadrantes), axis=1)
#casas = pd.get_dummies(casas, columns=['regiao'], prefix='regiao')

#casas = casas.drop('lat', axis=1)
#casas = casas.drop('long', axis=1)

In [279]:
scaler = MinMaxScaler()

features = casas.drop("price", axis=1)
target = casas["price"]

In [280]:
X_train, X_test, y_train, y_test = train_test_split(features.copy(), target.copy(), test_size=0.30, random_state=50)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = XGBRegressor(colsample_bytree=0.2, gamma=0.0, learning_rate=0.01, max_depth=7, min_child_weight=1.5, n_estimators=7200, reg_alpha=0.9, reg_lambda=0.6, subsample=0.2, seed=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'R^2 no conjunto de teste: {r2_test}')
print(f'R^2 no conjunto de treino: {r2_train}')

R^2 no conjunto de teste: 0.947017489005995
R^2 no conjunto de treino: 0.9888591375614149


In [281]:
def create_model(data, target, algo, scaler=None):
	if scaler != None:
		X = scaler.fit_transform(data)
	else:
		X = data
	y = target
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=50)
	model = algo
	model.fit(X_train, y_train)
	return(model, X_train, X_test, y_train, y_test)

# Lista de modelos
modelos = [RandomForestRegressor(), LinearRegression(), GradientBoostingRegressor(), KNeighborsRegressor(), Lasso(), Ridge(), ElasticNet()]

for modelo in modelos:
    modelo, X_train, X_test, y_train, y_test = create_model(features.copy(), target.copy(), modelo, scaler=None)

    # Fazendo previsões
    y_pred = modelo.predict(X_test)
    y_pred_train = modelo.predict(X_train)

    # Calculando métricas
    mse = mean_squared_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    r2_train = r2_score(y_train, y_pred_train)

    # Imprimindo métricas
    print(f'Modelo: {modelo.__class__.__name__}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'R2 (Test) : {r2_test}')
    print(f'R2 (Training) : {r2_train}')
    print('-'*50)

Modelo: RandomForestRegressor
Mean Squared Error (MSE): 6592582248.176868
R2 (Test) : 0.9520915021834695
R2 (Training) : 0.9923562613842625
--------------------------------------------------
Modelo: LinearRegression
Mean Squared Error (MSE): 30120105253.338192
R2 (Test) : 0.7811162694007683
R2 (Training) : 0.7745235762777538
--------------------------------------------------
Modelo: GradientBoostingRegressor
Mean Squared Error (MSE): 10209056265.219181
R2 (Test) : 0.9258104743514808
R2 (Training) : 0.936997852175775
--------------------------------------------------
Modelo: KNeighborsRegressor
Mean Squared Error (MSE): 25815796000.055573
R2 (Test) : 0.8123958170347154
R2 (Training) : 0.8800712222157036
--------------------------------------------------
Modelo: Lasso
Mean Squared Error (MSE): 30120304068.045044
R2 (Test) : 0.7811148246081832
R2 (Training) : 0.7745235735383446
--------------------------------------------------
Modelo: Ridge
Mean Squared Error (MSE): 30123629589.15392
R2 